# Procesamiento de los datos

## Importar librerías y Cargar datos

In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import seaborn as sb
from sklearn.impute import SimpleImputer
import glob

In [27]:
# Cargar datos CNC
# Buscar todos los archivos que empiecen con 'd' y terminen en '.xlsx'
files = glob.glob("d*.xlsx")

# Ordenar los archivos por año (por si no vienen en orden)
files = sorted(files)

# Crear un dataframe por cada año con nombre CNC{AÑO}
for f in files:
    year = f.split(".")[0][1:]  # extrae el año, ej: d2018.xlsx -> "2018"
    globals()[f"CNC{year}"] = pd.read_excel(f)

In [28]:
# Cargar datos ICFES
cols = [
    "periodo", "estu_consecutivo", "estu_genero",
    "estu_cod_reside_mcpio", "estu_mcpio_reside",
    "estu_pais_reside", "fami_estratovivienda", "fami_tieneinternet"
]

files = glob.glob("Examen_Saber_11_*.txt")
icfes_data = {}

for f in files:
    name = f.replace("Examen_Saber_11_", "").replace(".txt", "")
    year, semester = name[:4], name[4:]
    df = pd.read_csv(f, delimiter=";")[cols]
    df.columns = df.columns.str.upper()
    icfes_data[(year, semester)] = df

# 🔗 Concatenar todos en un solo DataFrame
ICFES = pd.concat(icfes_data.values(), ignore_index=True)

/tmp/ipython-input-2163999175.py:14: DtypeWarning: Columns (4,5,6,7,13,14,15,16,17,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f, delimiter=";")[cols]
/tmp/ipython-input-2163999175.py:14: DtypeWarning: Columns (4,5,6,7,13,14,15,16,17,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f, delimiter=";")[cols]
/tmp/ipython-input-2163999175.py:14: DtypeWarning: Columns (4,5,6,7,13,14,15,16,17,18,19,20,31,32,41,47,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f, delimiter=";")[cols]
/tmp/ipython-input-2163999175.py:14: DtypeWarning: Columns (4,5,6,7,13,14,15,16,17,18,19,20,31,32,40,46,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f, delimiter=";")[cols]


In [29]:
# Cargar datos MEN
MEN = pd.read_csv("MEN.csv", delimiter=",")
MEN = MEN[["AÑO", "CÓDIGO_MUNICIPIO", "MUNICIPIO", "POBLACIÓN_5_16", "COBERTURA_NETA"]]

## Procesar datos originales

### CNC

In [30]:

selected_columns_2018 = [
    # Bloque 1: Geográficas
    'LAT_1', 'LONG_1', 'SECTOR', 'SECCION', 'MANZANA',

    # Bloque 2: # Sociodemográficas
    'EDAD', 'RANGO_EDAD', 'DH', 'PA', 'PB', 'PD', 'PB1', 'SEXO', 'ESTRATO',

    # Bloque 3: Administrativas
    'ID', 'FECHAINI', 'FECHAFIN', 'DIREC_VIVI', 'ESTUDIO', 'ENTREVISTADOR',

    # Bloque 4: Uso de internet
    'P7', 'P8', 'P9', 'P10', 'P11',

    # Bloque 5: Equipamiento de la vivienda
    'P15', 'P16_1', 'P16_2', 'P16_3', 'P16_4', 'P16_5', 'P16_6', 'P16_7',
    'P16_8', 'P16_9', 'P16_10', 'P16_11', 'P17', 'P19', 'P21',

    # Bloque 6: Conectividad del hogar
    'P27', 'P29',

    # Bloque 8: Percepciones sobre internet
    'P33', 'P34', 'P35', 'P36_1', 'P36_2', 'P36_3', 'P36_4',

    # Bloque 9: Comercio electrónico
    'P56', 'P57', 'P58', 'P59', 'P64', 'NIVEL_PIRAMIDEAJUSTA'
]
CNC2018_col_sel = CNC2018[selected_columns_2018]

selected_columns_2023 = [
    'GPSLONG',  # Longitud GPS
    'GPSLAT',   # Latitud GPS
    'GPSALT',   # Altitud GPS
    'GPSPRECISION',  # Precisión del GPS
    'REGION_BASE',  # Región base
    'REGIONAL',  # Regional
    'MUNICIPIO_BASE',  # Municipio base
    'MUNICIPIO',  # Municipio
    'DANE',  # Código DANE
    'POINT_X',  # Coordenada X
    'POINT_Y',  # Coordenada Y
    'ESTRATO',  # Estrato socioeconómico
    'SECTOR',  # Sector
    'SECCION',  # Sección
    'MANZANA',  # Manzana
    'PB1',
    'TIPO_ENCUESTA',  # Tipo de encuesta
    'TIPO',  # Tipo de vivienda o encuesta
    'GENERO',  # Género del entrevistado
    'EDAD',  # Edad del entrevistado
    'PERSONAS',  # Número de personas en el hogar
    'PERSONAS_GEN',  # Distribución de género por personas
    'GENERO_SEL',  # Género de la persona seleccionada
    'REDAD',  # Rango de edad
    'ALEATORIO_PDET',  # Indicador de muestreo aleatorio en zonas PDET
    'VERIFICA',  # Validación o verificación del encuestado
    'DIRECCION_FILTRO',  # Dirección registrada
    'B4_1', #Pregunta B4.1 Formulario
    'B4_2', #Pregunta B4.2 Formulario
    'B4_4_TOTAL', #Pregunta B4.4 Formulario
    'B4_5_TOTAL', #Pregunta B4.5 Formulario
    'B4_7', #Pregunta B4.7 Formulario
    'B5_1', 'B5_2_1', 'B5_2_2', 'B5_2_3', 'B5_2_4', 'B5_2_5', 'B5_2_6', 'B5_2_7',
    'B5_2_8', 'B5_2_9', 'B5_2_10', 'B5_2_11', 'B5_2_12', 'B5_3_1', 'B5_3_2', 'B5_3_3',
    'B5_3_4', 'B5_3_5', 'B5_3_6', 'B5_3_7', 'B5_3_8', 'B5_3_9', 'B5_3_10', 'B5_3_11',
    'B5_3_12', 'B5_6', 'B5_8', 'B6_1', 'B6_3', 'B7_1', 'B7_2', 'B8_1_1', 'B8_1_2', 'B8_1_3',
    'B8_1_4', 'B8_1_5', 'B8_1_6', 'B8_1_7', 'B8_1_8', 'B8_1_9', 'B8_2_1', 'B8_2_2',
    'B8_2_3', 'B8_2_4', 'B8_2_5', 'B8_2_6', 'B8_2_7', 'B8_2_8', 'B8_2_9', 'B8_2_10',
    'B8_2_11', 'B8_2_12', 'B8_2_13', 'B8_2_14', 'B8_2_15', 'B8_2_16', 'B8_2_17',
    'B8_2_18', 'B8_2_19', 'B8_2_20', 'B8_2_21', 'B8_2_22', 'B9_1', 'B9_2', 'nivel_piramide', 'B9_7'
]

CNC2023_col_sel = CNC2023[selected_columns_2023]

### MEN

In [31]:
# Eliminar registros con columnas vacías
MEN =  MEN[MEN.isnull().sum(axis=1) <= 1]
MEN.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14579 entries, 0 to 14584
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   AÑO               14579 non-null  int64 
 1   CÓDIGO_MUNICIPIO  14579 non-null  int64 
 2   MUNICIPIO         14579 non-null  object
 3   POBLACIÓN_5_16    14579 non-null  object
 4   COBERTURA_NETA    14474 non-null  object
dtypes: int64(2), object(3)
memory usage: 683.4+ KB


In [32]:
# Asegurar que COBERTURA_NETA quede como número sin %
MEN["COBERTURA_NETA"] = (
    MEN["COBERTURA_NETA"]
    .astype(str)                      # forzar a string
    .str.replace("%", "", regex=False) # quitar %
    .str.strip()                       # quitar espacios
    .replace("", "0")                  # manejar vacíos
    .astype(float)                     # a número
)

In [33]:
# Asegurar que POBLACIÓN_5_16 también sea numérico
MEN["POBLACIÓN_5_16"] = (
    MEN["POBLACIÓN_5_16"]
    .astype(str)
    .str.replace(",", "", regex=False)
    .str.strip()
    .replace("", "0")
    .astype(float)
)

In [34]:
MEN["Estudiantes_5_16"] = (
    MEN["POBLACIÓN_5_16"] * (MEN["COBERTURA_NETA"] / 100)
).round(0)

In [35]:
# Agrupar por municipio y año
MEN = MEN.groupby(["CÓDIGO_MUNICIPIO", "MUNICIPIO", "AÑO"], as_index=False).agg(
    POBLACION_5_16=("POBLACIÓN_5_16", "sum"),
    Estudiantes_5_16=("Estudiantes_5_16", "sum"))

### ICFES

In [36]:
# Eliminar registros de estudiantes que residen por fuera de Colombia
ICFES = ICFES[ICFES['ESTU_PAIS_RESIDE'] == 'COLOMBIA']
ICFES.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1343561 entries, 0 to 1354637
Data columns (total 8 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   PERIODO                1343561 non-null  int64  
 1   ESTU_CONSECUTIVO       1343561 non-null  object 
 2   ESTU_GENERO            1343512 non-null  object 
 3   ESTU_COD_RESIDE_MCPIO  1341698 non-null  float64
 4   ESTU_MCPIO_RESIDE      1341698 non-null  object 
 5   ESTU_PAIS_RESIDE       1343561 non-null  object 
 6   FAMI_ESTRATOVIVIENDA   1056825 non-null  object 
 7   FAMI_TIENEINTERNET     1065562 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 92.3+ MB


In [37]:
# Eliminar periodos anteriores a 2012
periodos_a_eliminar = [20102, 20111, 20112, 20101]
ICFES = ICFES[~ICFES['PERIODO'].isin(periodos_a_eliminar)]

In [38]:
# Eliminar registros con más de 2 columnas vacías
ICFES = ICFES[ICFES.isnull().sum(axis=1) <= 2]

In [39]:
# Eliminar registros con exactamente 2 columnas faltantes
ICFES = ICFES[ICFES.isnull().sum(axis=1) != 2]

In [40]:
# Mostrar registros con al menos un campo faltante
registros_con_faltantes = ICFES[ICFES.isnull().any(axis=1)]

In [41]:
# Imputación de la variable 'ESTU_GENERO' con la moda
# Calcular la moda de la columna 'ESTU_GENERO'
moda_genero = ICFES['ESTU_GENERO'].mode()[0]

# Reemplazar los valores faltantes con la moda
ICFES['ESTU_GENERO'].fillna(moda_genero, inplace=True)

# Verificar si la imputación fue exitosa
print(ICFES['ESTU_GENERO'].isnull().sum())

0


/tmp/ipython-input-699813981.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  ICFES['ESTU_GENERO'].fillna(moda_genero, inplace=True)


In [42]:
# Imputar 'FAMI_ESTRATOVIVIENDA' y 'FAMI_TIENEINTERNET' por la moda dentro de cada grupo de 'ESTU_COD_RESIDE_MCPIO'
for columna in ['FAMI_ESTRATOVIVIENDA', 'FAMI_TIENEINTERNET']:
    # Imputación por la moda dentro de cada grupo
    ICFES[columna] = ICFES.groupby('ESTU_COD_RESIDE_MCPIO')[columna].transform(lambda x: x.fillna(x.mode()[0]))

# Verificación
print(ICFES[['FAMI_ESTRATOVIVIENDA', 'FAMI_TIENEINTERNET']].isnull().sum())

FAMI_ESTRATOVIVIENDA    0
FAMI_TIENEINTERNET      0
dtype: int64


In [43]:
# Eliminar ESTU_PAIS_RESIDE
ICFES = ICFES.drop('ESTU_PAIS_RESIDE', axis=1)
ICFES.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1068706 entries, 0 to 1347154
Data columns (total 7 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   PERIODO                1068706 non-null  int64  
 1   ESTU_CONSECUTIVO       1068706 non-null  object 
 2   ESTU_GENERO            1068706 non-null  object 
 3   ESTU_COD_RESIDE_MCPIO  1068706 non-null  float64
 4   ESTU_MCPIO_RESIDE      1068706 non-null  object 
 5   FAMI_ESTRATOVIVIENDA   1068706 non-null  object 
 6   FAMI_TIENEINTERNET     1068706 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 65.2+ MB


In [44]:
# Codificar acceso
ICFES["FAMI_TIENEINTERNET"] = ICFES["FAMI_TIENEINTERNET"].map({"Si": 1, "No": 0}).astype(int)

In [45]:
# Crear variable año
ICFES["Año"] = ICFES["PERIODO"].astype(str).str[:4].astype(int)

In [46]:
# Agrupar por municipio y año
ICFES = ICFES.groupby(["ESTU_COD_RESIDE_MCPIO", "ESTU_MCPIO_RESIDE", "Año"], as_index=False).agg(
    HOGARES_INTERNET=("FAMI_TIENEINTERNET", "sum"),
    POBLACIÓN_ICFES=("FAMI_TIENEINTERNET", "count"))

## Consolidar datos

In [47]:
def to_dane5(x):
    """Convierte a string de 5 dígitos (p.ej. 5001 -> '05001', 11001 -> '11001')."""
    if pd.isna(x):
        return pd.NA
    try:
        s = str(int(float(x)))
    except Exception:
        s = str(x).strip()
        s = ''.join(ch for ch in s if ch.isdigit())
    return s.zfill(5)

def map_genero(val):
    """Normaliza género a 1=Masculino, 2=Femenino (admite códigos o texto)."""
    if pd.isna(val):
        return pd.NA
    v = str(val).strip().upper()
    if v in {"1", "M", "H", "HOMBRE", "MASCULINO"}:
        return 1
    if v in {"2", "F", "MUJER", "FEMENINO"}:
        return 2
    # En CNC 2023 ya viene 1-2 intenta convertir
    try:
        n = int(float(v))
        if n in (1, 2):
            return n
    except Exception:
        pass
    return pd.NA

In [48]:
# 1) CNC como BASE (2018 y 2023)
# ---- 2018
cnc18 = CNC2018_col_sel.copy()

# Llave municipio
cnc18["DANE5"] = cnc18["PB"].apply(to_dane5)
cnc18["AÑO"] = 2018

# Normalización de nombres y campos
cnc18 = cnc18.rename(columns={
    "LAT_1": "GPSLAT",
    "LONG_1": "GPSLONG",
    "RANGO_EDAD": "REDAD",
    "DIREC_VIVI": "DIRECCION_FILTRO",
    "NIVEL_PIRAMIDEAJUSTA": "NIVEL_PIRAMIDE",
    "PD": "PERSONAS"
})

# Genero desde SEXO -> GENERO (1=M, 2=F)
cnc18["GENERO"] = cnc18["SEXO"].apply(map_genero)

# Campos que no existen en 2018 (para homogeneidad con 2023)
for col_abs in ["GPSALT", "GPSPRECISION", "POINT_X", "POINT_Y",
                "REGION_BASE", "REGIONAL", "MUNICIPIO_BASE",
                "TIPO_ENCUESTA", "TIPO", "PERSONAS", "PERSONAS_GEN",
                "GENERO_SEL", "ALEATORIO_PDET", "VERIFICA", "DANE"]:
    if col_abs not in cnc18.columns:
        cnc18[col_abs] = pd.NA

# Asegurar DANE numérico tipo texto según diccionario
cnc18["DANE"] = cnc18["DANE5"]

# 2023
cnc23 = CNC2023_col_sel.copy()
cnc23["AÑO"] = 2023
cnc23["DANE5"] = cnc23["DANE"].apply(to_dane5)

# Normalizaciones
cnc23 = cnc23.rename(columns={
    "nivel_piramide": "NIVEL_PIRAMIDE",
    'B4_1': 'P7',
    'B4_4_TOTAL': 'P10',
    'B5_1': 'P15',
    'B6_1': 'P27',
    'B6_3': 'P29',
    'B7_1': 'P33',
    'B7_2': 'P34',
    'B9_1': 'P56',
    'B9_3': 'P57',
    'B9_7': 'P64',
    'B4_2': 'P9'
})
# Genero ya es 1-2, map_genero
cnc23["GENERO"] = cnc23["GENERO"].apply(map_genero)

# Unir bases CNC
CNC_BASE = pd.concat([cnc18, cnc23], ignore_index=True, sort=False)

# Eliminar columnas que no coinciden en ambas bases o son irrelevantes
cols_to_drop = [
    "P16_1", "P16_2", "P16_3", "P16_4", "P16_5", "P16_6", "P16_7", "P16_8", "P16_9", "P16_10", "P16_11",
    "P17", "P19", "P21", "P35", "P36_1", "P36_2", "P36_3", "P36_4", "P58", "P59",
    "B5_3_4", "B5_3_12", "B8_1_4", "B8_2_3", "B8_2_11", "B8_2_18",
    "B5_3_5", "B5_6", "B8_1_5", "B8_2_4", "B8_2_12", "B8_2_19",
    "B5_3_6", "B5_8", "B8_1_6", "B8_2_5", "B8_2_13", "B8_2_20",
    "B5_2_11", "B5_3_7", "B8_1_7", "B8_2_6", "B8_2_14", "B8_2_21",
    "B5_2_12", "B5_3_8", "B8_1_8", "B8_2_7", "B8_2_15", "B8_2_22",
    "B5_3_1", "B5_3_9", "B8_1_1", "B8_1_9", "B8_2_8", "B8_2_16",
    "B5_3_2", "B5_3_10", "B8_1_2", "B8_2_1", "B8_2_9", "B8_2_17", "B9_2",
    "B5_3_3", "B5_3_11", "B8_1_3", "B8_2_2", "B8_2_10",  "B5_2_1", "B5_2_2", "B5_2_3", "B5_2_4", "B5_2_5",
    "B5_2_6", "B5_2_7", "B5_2_8", "B5_2_9", "B5_2_10", "DH", "PA", "FECHAINI", "FECHAFIN", "ESTUDIO", "ENTREVISTADOR", "VERIFICA",
    "ALEATORIO_PDET", "SEXO", "GENERO_SEL", "GPSALT", "GPSPRECISION", "POINT_X", "POINT_Y", "REGION_BASE", "REGIONAL", "TIPO_ENCUESTA",
    "TIPO", "PERSONAS_GEN", "B4_5_TOTAL", "B4_7", "ID", "P8", "P11", "MUNICIPIO_BASE", "MUNICIPIO", "DANE", "PB"
]

CNC_BASE = CNC_BASE.drop(columns=cols_to_drop)

/tmp/ipython-input-1047317945.py:57: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  CNC_BASE = pd.concat([cnc18, cnc23], ignore_index=True, sort=False)


In [49]:
orden_columnas = [
    "AÑO", "GPSLAT", "GPSLONG", "SECTOR", "SECCION", "MANZANA",
    "DIRECCION_FILTRO", "DANE5", "PB1", "PERSONAS", "EDAD", "REDAD",
    "GENERO", "ESTRATO", "P7", "P9", "P10", "P15", "P27", "P29",
    "P33", "P34", "P56", "P57", "P64", "NIVEL_PIRAMIDE"
]

CNC_BASE = CNC_BASE[orden_columnas]

In [50]:
# 2) MEN (filtrar 2018/2023 y armar KPIs)
# MEN: ['CÓDIGO_MUNICIPIO','MUNICIPIO','AÑO','POBLACION_5_16','Estudiantes_5_16']

men = MEN.copy()
men = men[men["AÑO"].isin([2018, 2023])].copy()
men["DANE5"] = men["CÓDIGO_MUNICIPIO"].apply(to_dane5)
men["PROP_EDUC_5_16_MEN"] = (men["Estudiantes_5_16"] / men["POBLACION_5_16"]).replace([np.inf, -np.inf], np.nan)

In [51]:
# Diccionario DANE5 --> nombre municipio
mpio_nombres = men[["DANE5", "MUNICIPIO"]].drop_duplicates(subset=["DANE5"]).rename(columns={"MUNICIPIO": "MUNICIPIO_NOMBRE"})

men_kpis = men[["DANE5", "AÑO", "POBLACION_5_16", "Estudiantes_5_16", "PROP_EDUC_5_16_MEN"]].copy()
assert not men_kpis.duplicated(["DANE5", "AÑO"]).any(), "MEN tiene llaves duplicadas DANE5-AÑO"

# 3) ICFES (ya agregado por municipio-año)
# ICFES: ['ESTU_COD_RESIDE_MCPIO','ESTU_MCPIO_RESIDE','Año','HOGARES_INTERNET','POBLACIÓN_ICFES']

icfes = ICFES.copy()
icfes = icfes[icfes["Año"].isin([2018, 2023])].copy()
icfes["DANE5"] = icfes["ESTU_COD_RESIDE_MCPIO"].apply(to_dane5)
icfes = icfes.rename(columns={"Año": "AÑO"})
icfes["TASA_INTERNET_ICFES"] = (icfes["HOGARES_INTERNET"] / icfes["POBLACIÓN_ICFES"]).replace([np.inf, -np.inf], np.nan)

icfes_kpis = icfes[["DANE5", "AÑO", "HOGARES_INTERNET", "POBLACIÓN_ICFES", "TASA_INTERNET_ICFES"]].copy()
assert not icfes_kpis.duplicated(["DANE5", "AÑO"]).any(), "ICFES tiene llaves duplicadas DANE5-AÑO"

# 4) MERGE sobre CNC (left) y agregar nombre de municipio

cnc_enriquecido = (
    CNC_BASE
    .merge(men_kpis, on=["DANE5", "AÑO"], how="left")
    .merge(icfes_kpis, on=["DANE5", "AÑO"], how="left")
    .merge(mpio_nombres, on="DANE5", how="left")
)

# Cobertura de los merges
print("Filas CNC_BASE:", len(CNC_BASE))
print("Cobertura MEN:", cnc_enriquecido["POBLACION_5_16"].notna().mean().round(3))
print("Cobertura ICFES:", cnc_enriquecido["POBLACIÓN_ICFES"].notna().mean().round(3))

# 5) Panel municipal (agregado municipio-año)
#    — métricas de CNC (conteo, edad promedio, % mujeres) + MEN + ICFES

def pct_mujeres(s):
    s = pd.to_numeric(s, errors="coerce")
    return np.nan if s.isna().all() else (s.eq(2).mean())

cnc_mpio = (
    cnc_enriquecido
    .groupby(["DANE5", "MUNICIPIO_NOMBRE", "AÑO"], as_index=False)
    .agg(
        ENCUESTAS_CNC=("AÑO", "size"),
        EDAD_PROM=("EDAD", "mean"),
        PROP_MUJERES=("GENERO", pct_mujeres),
        NIVEL_PIRAMIDE_PROM=("NIVEL_PIRAMIDE", "mean")
    )
)

panel_mpio = (
    cnc_mpio
    .merge(men_kpis, on=["DANE5", "AÑO"], how="left")
    .merge(icfes_kpis, on=["DANE5", "AÑO"], how="left")
)

# 6) Columnas clave para revisión rápida

cols_cnc = [c for c in [
    "AÑO","DANE5","MUNICIPIO_NOMBRE","GPSLAT","GPSLONG","EDAD","REDAD","GENERO",
    "NIVEL_PIRAMIDE","SECTOR","SECCION","MANZANA",
    "POBLACION_5_16","Estudiantes_5_16","PROP_EDUC_5_16_MEN",
    "HOGARES_INTERNET","POBLACIÓN_ICFES","TASA_INTERNET_ICFES"
] if c in cnc_enriquecido.columns]

cols_panel = [c for c in [
    "AÑO","DANE5","MUNICIPIO_NOMBRE",
    "ENCUESTAS_CNC","EDAD_PROM","PROP_MUJERES","NIVEL_PIRAMIDE_PROM",
    "POBLACION_5_16","Estudiantes_5_16","PROP_EDUC_5_16_MEN",
    "HOGARES_INTERNET","POBLACIÓN_ICFES","TASA_INTERNET_ICFES"
] if c in panel_mpio.columns]

print("\nVista rápida CNC enriquecido:")
display(cnc_enriquecido[cols_cnc])

# print("\nVista rápida Panel municipal:")
# display(panel_mpio[cols_panel].head(10))

# (Opcional) Guardar
# cnc_enriquecido.to_parquet("data/cnc_enriquecido_2018_2023.parquet", index=False)
# panel_mpio.to_parquet("data/panel_mpio_2018_2023.parquet", index=False)

Filas CNC_BASE: 7662
Cobertura MEN: 1.0
Cobertura ICFES: 1.0

Vista rápida CNC enriquecido:


,AÑO,DANE5,MUNICIPIO_NOMBRE,GPSLAT,GPSLONG,EDAD,REDAD,GENERO,NIVEL_PIRAMIDE,SECTOR,SECCION,MANZANA,POBLACION_5_16,Estudiantes_5_16,PROP_EDUC_5_16_MEN,HOGARES_INTERNET,POBLACIÓN_ICFES,TASA_INTERNET_ICFES
0,2018,08001,Barranquilla,10.952493,-74.777018,69,7,2,0,2201,8,3,233015.0,235368.0,1.010098,11425,16359,0.698392
1,2018,05001,Medellín,NaN,NaN,56,6,2,0,203,1,13,384358.0,372366.0,0.968800,21663,26759,0.809559
2,2018,05001,Medellín,NaN,NaN,64,6,2,0,203,1,13,384358.0,372366.0,0.968800,21663,26759,0.809559
3,2018,05001,Medellín,6.299582,-75.550590,38,4,1,1,203,2,2,384358.0,372366.0,0.968800,21663,26759,0.809559
4,2018,68001,Bucaramanga,7.090175,-73.145032,43,4,1,2,798,2,25,94655.0,95781.0,1.011896,5355,6446,0.830748
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7657,2023,25175,Chía,4.843900,74.066290,69,7,1,2,1,3,19,24997.0,27437.0,1.097612,1715,1839,0.932572
7658,2023,25175,Chía,NaN,NaN,24,2,1,1,1,2,7,24997.0,27437.0,1.097612,1715,1839,0.932572
7659,2023,25175,Chía,4.857498,74.055293,47,5,1,3,1,7,5,24997.0,27437.0,1.097612,1715,1839,0.932572
7660,2023,25175,Chía,4.853827,74.056633,58,6,2,1,2,6,35,24997.0,27437.0,1.097612,1715,1839,0.932572


In [52]:
# Agregar nombre municipio

DIVIPOLA = pd.read_csv("DIVIPOLA.csv", delimiter=",")
DANE = DIVIPOLA.copy()
DANE["DANE5"] = DIVIPOLA["Código Municipio"].apply(to_dane5)

In [53]:
Completo = cnc_enriquecido.merge(
    DANE[["DANE5", "Nombre Municipio", "Nombre Departamento"]],
    on="DANE5",
    how="left"
)

In [54]:
# Carga opcional de datos si no existe df_data
df_data = Completo

## Tratamiento de datos final

se crean variables dummies de la pregunta P7

In [55]:
df_data['P7'].unique()

array([   9,    1,    2, 1234,   12,  123,    3,   21,   31,  124,   13,
         14,   32,  134,  132,   23,  431,  213,  321, 1324,    4,  231,
       1243,   43, 3241,  341,  241,  312, 1432, 3412,   24, 2341, 2134,
         41,  324, 3124,  423, 3421, 1342,  314,  142,  421, 4123, 3214,
        432,   88, 3142,   42,  412,  143,  214,  342, 4231, 1423])

Función para manejar las categorías

In [56]:
def create_dummies_with_nan_handling(df, column_name, special_cases=None):
    """
    Crea columnas binarias (dummies) para cada número individual único en una columna,
    manejando casos especiales y dejando NaN en las nuevas columnas si el valor original es NaN.

    Args:
        df (pd.DataFrame): DataFrame original.
        column_name (str): Nombre de la columna a procesar.
        special_cases (list): Lista de valores que deben tratarse como casos completos (e.g., '88').

    Returns:
        pd.DataFrame: DataFrame con las columnas dummies agregadas.
    """
    if special_cases is None:
        special_cases = []

    # Crear un conjunto para almacenar cada número individual y los casos especiales
    all_numbers = set(special_cases)

    # Recorrer los valores únicos de la columna
    unique_values = df[column_name].dropna().astype(str).unique()

    for value in unique_values:
        if value in special_cases:
            all_numbers.add(value)  # Mantener el caso especial completo
        else:
            for char in value:
                if char.isdigit():  # Solo considerar caracteres numéricos individuales
                    all_numbers.add(char)

    # Convertir el conjunto en una lista ordenada
    all_numbers = sorted(all_numbers, key=lambda x: int(x))

    # Crear columnas dummies para cada número individual y casos especiales
    for number in all_numbers:
        dummy_name = f"{column_name}_{number}"
        df[dummy_name] = df[column_name].apply(
            lambda x: np.nan if pd.isna(x) else int(number in str(x))
        )
    df = df.drop(columns=[column_name])
    return df


Se crean las variables dummies

In [57]:
df_data_1 = create_dummies_with_nan_handling(df_data, "P7", special_cases=["9", "88"])
df_data_1['Servicios_Telecomunicaciones_No'] = df_data_1['P7_9']|df_data_1['P7_88']
df_data_1['Servicios_Telecomunicaciones_Si'] = df_data_1['P7_1']|df_data_1['P7_2']|df_data_1['P7_3']|df_data_1['P7_4']
df_data_1 = df_data_1.drop(['P7_9','P7_88','P7_1','P7_2','P7_3','P7_4'],axis=1)
df_data_1

,AÑO,GPSLAT,GPSLONG,SECTOR,SECCION,MANZANA,DIRECCION_FILTRO,DANE5,PB1,PERSONAS,...,Estudiantes_5_16,PROP_EDUC_5_16_MEN,HOGARES_INTERNET,POBLACIÓN_ICFES,TASA_INTERNET_ICFES,MUNICIPIO_NOMBRE,Nombre Municipio,Nombre Departamento,Servicios_Telecomunicaciones_No,Servicios_Telecomunicaciones_Si
0,2018,10.952493,-74.777018,2201,8,3,Kr 14 # 21 105,08001,1,1,...,235368.0,1.010098,11425,16359,0.698392,Barranquilla,BARRANQUILLA,ATLÁNTICO,1,0
1,2018,NaN,NaN,203,1,13,kra 46A # 110_64,05001,1,5,...,372366.0,0.968800,21663,26759,0.809559,Medellín,MEDELLÍN,ANTIOQUIA,0,1
2,2018,NaN,NaN,203,1,13,Kra 46A # 110_60,05001,1,4,...,372366.0,0.968800,21663,26759,0.809559,Medellín,MEDELLÍN,ANTIOQUIA,0,1
3,2018,6.299582,-75.550590,203,2,2,kra 109 201 _28,05001,1,2,...,372366.0,0.968800,21663,26759,0.809559,Medellín,MEDELLÍN,ANTIOQUIA,0,1
4,2018,7.090175,-73.145032,798,2,25,Cll 58 47 aw 8,68001,1,4,...,95781.0,1.011896,5355,6446,0.830748,Bucaramanga,BUCARAMANGA,SANTANDER,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7657,2023,4.843900,74.066290,1,3,19,Calle14 13 80,25175,1,5,...,27437.0,1.097612,1715,1839,0.932572,Chía,CHÍA,CUNDINAMARCA,0,1
7658,2023,NaN,NaN,1,2,7,Cl 6 a 3 99,25175,1,4,...,27437.0,1.097612,1715,1839,0.932572,Chía,CHÍA,CUNDINAMARCA,0,1
7659,2023,4.857498,74.055293,1,7,5,Calle 10 4 48 capincito,25175,1,5,...,27437.0,1.097612,1715,1839,0.932572,Chía,CHÍA,CUNDINAMARCA,0,1
7660,2023,4.853827,74.056633,2,6,35,Cea 3 6 20,25175,1,3,...,27437.0,1.097612,1715,1839,0.932572,Chía,CHÍA,CUNDINAMARCA,0,1


Se crea las variables dummies de la pregunta P15

In [58]:
df_data['P15'].unique()

array(['99', '02', '010304', '03', '0203', '060701', '0103', '01030405',
       '0204030810', '01', '0103050608', '06', '0102', '01030405060708',
       '04', '0305', '0403', '0304', '020409', '0408', '020403', '010203',
       '01080903', '0102030406', '010307', '010203040809', '020305',
       '020306', '0203040708', '020304', '0205', '01030406', '030608',
       '05', '040203', '01020304050608', '010204', '0405', '010306',
       '0204', '010203040608', '0103050607', '0603', '0307', '02030406',
       '0102031005', '020305040610', '010305', '01020305', '0102030405',
       '020308', '010203040607', '01030507', '010203050608',
       '0204030506071011', '01020306', '050403', '030405', '0302',
       '01030508', '01020405', '0206', '010402', '0205060308', '030406',
       '0102030507080910', '010406', '0104', '0301020405', '0306',
       '060304', '0203050607', '02030508', '010205', '02030504',
       '02030405', '02030409', '02030405080910', '0501', '050103',
       '030206', '0301',

In [59]:
df_data_1['Dispositivos_hogar_No'] = df_data_1['P15']=="99"
df_data_1= df_data_1.drop(['P15'], axis=1)
df_data_1

,AÑO,GPSLAT,GPSLONG,SECTOR,SECCION,MANZANA,DIRECCION_FILTRO,DANE5,PB1,PERSONAS,...,PROP_EDUC_5_16_MEN,HOGARES_INTERNET,POBLACIÓN_ICFES,TASA_INTERNET_ICFES,MUNICIPIO_NOMBRE,Nombre Municipio,Nombre Departamento,Servicios_Telecomunicaciones_No,Servicios_Telecomunicaciones_Si,Dispositivos_hogar_No
0,2018,10.952493,-74.777018,2201,8,3,Kr 14 # 21 105,08001,1,1,...,1.010098,11425,16359,0.698392,Barranquilla,BARRANQUILLA,ATLÁNTICO,1,0,True
1,2018,NaN,NaN,203,1,13,kra 46A # 110_64,05001,1,5,...,0.968800,21663,26759,0.809559,Medellín,MEDELLÍN,ANTIOQUIA,0,1,True
2,2018,NaN,NaN,203,1,13,Kra 46A # 110_60,05001,1,4,...,0.968800,21663,26759,0.809559,Medellín,MEDELLÍN,ANTIOQUIA,0,1,True
3,2018,6.299582,-75.550590,203,2,2,kra 109 201 _28,05001,1,2,...,0.968800,21663,26759,0.809559,Medellín,MEDELLÍN,ANTIOQUIA,0,1,False
4,2018,7.090175,-73.145032,798,2,25,Cll 58 47 aw 8,68001,1,4,...,1.011896,5355,6446,0.830748,Bucaramanga,BUCARAMANGA,SANTANDER,0,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7657,2023,4.843900,74.066290,1,3,19,Calle14 13 80,25175,1,5,...,1.097612,1715,1839,0.932572,Chía,CHÍA,CUNDINAMARCA,0,1,False
7658,2023,NaN,NaN,1,2,7,Cl 6 a 3 99,25175,1,4,...,1.097612,1715,1839,0.932572,Chía,CHÍA,CUNDINAMARCA,0,1,False
7659,2023,4.857498,74.055293,1,7,5,Calle 10 4 48 capincito,25175,1,5,...,1.097612,1715,1839,0.932572,Chía,CHÍA,CUNDINAMARCA,0,1,False
7660,2023,4.853827,74.056633,2,6,35,Cea 3 6 20,25175,1,3,...,1.097612,1715,1839,0.932572,Chía,CHÍA,CUNDINAMARCA,0,1,False


Se crea variables dummies para la pregunta P27

In [60]:
df_data_1['P27']=df_data['P27'].astype(str).apply(lambda x: x.strip(".0"))
df_data_1['P27'].unique()


array(['nan', '1', '3', '9', '2', '88', '99', '77', '12', '21', '7'],
      dtype=object)

In [61]:
def create_dummies (x):
    if "1" in x or "2" in x or "3" in x:
        return True
    else:
        return False

df_data_1["conexion_hogar_si"]=df_data_1["P27"].apply(create_dummies)
df_data_1 = df_data_1.drop(["P27"], axis=1)
df_data_1

,AÑO,GPSLAT,GPSLONG,SECTOR,SECCION,MANZANA,DIRECCION_FILTRO,DANE5,PB1,PERSONAS,...,HOGARES_INTERNET,POBLACIÓN_ICFES,TASA_INTERNET_ICFES,MUNICIPIO_NOMBRE,Nombre Municipio,Nombre Departamento,Servicios_Telecomunicaciones_No,Servicios_Telecomunicaciones_Si,Dispositivos_hogar_No,conexion_hogar_si
0,2018,10.952493,-74.777018,2201,8,3,Kr 14 # 21 105,08001,1,1,...,11425,16359,0.698392,Barranquilla,BARRANQUILLA,ATLÁNTICO,1,0,True,False
1,2018,NaN,NaN,203,1,13,kra 46A # 110_64,05001,1,5,...,21663,26759,0.809559,Medellín,MEDELLÍN,ANTIOQUIA,0,1,True,False
2,2018,NaN,NaN,203,1,13,Kra 46A # 110_60,05001,1,4,...,21663,26759,0.809559,Medellín,MEDELLÍN,ANTIOQUIA,0,1,True,False
3,2018,6.299582,-75.550590,203,2,2,kra 109 201 _28,05001,1,2,...,21663,26759,0.809559,Medellín,MEDELLÍN,ANTIOQUIA,0,1,False,True
4,2018,7.090175,-73.145032,798,2,25,Cll 58 47 aw 8,68001,1,4,...,5355,6446,0.830748,Bucaramanga,BUCARAMANGA,SANTANDER,0,1,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7657,2023,4.843900,74.066290,1,3,19,Calle14 13 80,25175,1,5,...,1715,1839,0.932572,Chía,CHÍA,CUNDINAMARCA,0,1,False,True
7658,2023,NaN,NaN,1,2,7,Cl 6 a 3 99,25175,1,4,...,1715,1839,0.932572,Chía,CHÍA,CUNDINAMARCA,0,1,False,True
7659,2023,4.857498,74.055293,1,7,5,Calle 10 4 48 capincito,25175,1,5,...,1715,1839,0.932572,Chía,CHÍA,CUNDINAMARCA,0,1,False,True
7660,2023,4.853827,74.056633,2,6,35,Cea 3 6 20,25175,1,3,...,1715,1839,0.932572,Chía,CHÍA,CUNDINAMARCA,0,1,False,True


crear variables dummies para pregunta P29

In [62]:
df_data_1['P29']=df_data['P29'].astype(str).apply(lambda x: x.strip(".0"))

In [63]:
def create_dummies_P29 (x):
    if "1" in x:
        return True
    else:
        return False

df_data_1["interrupciones_si"]=df_data_1["P29"].apply(create_dummies_P29)
df_data_1 = df_data_1.drop(["P29"], axis=1)
df_data_1

,AÑO,GPSLAT,GPSLONG,SECTOR,SECCION,MANZANA,DIRECCION_FILTRO,DANE5,PB1,PERSONAS,...,POBLACIÓN_ICFES,TASA_INTERNET_ICFES,MUNICIPIO_NOMBRE,Nombre Municipio,Nombre Departamento,Servicios_Telecomunicaciones_No,Servicios_Telecomunicaciones_Si,Dispositivos_hogar_No,conexion_hogar_si,interrupciones_si
0,2018,10.952493,-74.777018,2201,8,3,Kr 14 # 21 105,08001,1,1,...,16359,0.698392,Barranquilla,BARRANQUILLA,ATLÁNTICO,1,0,True,False,False
1,2018,NaN,NaN,203,1,13,kra 46A # 110_64,05001,1,5,...,26759,0.809559,Medellín,MEDELLÍN,ANTIOQUIA,0,1,True,False,False
2,2018,NaN,NaN,203,1,13,Kra 46A # 110_60,05001,1,4,...,26759,0.809559,Medellín,MEDELLÍN,ANTIOQUIA,0,1,True,False,False
3,2018,6.299582,-75.550590,203,2,2,kra 109 201 _28,05001,1,2,...,26759,0.809559,Medellín,MEDELLÍN,ANTIOQUIA,0,1,False,True,False
4,2018,7.090175,-73.145032,798,2,25,Cll 58 47 aw 8,68001,1,4,...,6446,0.830748,Bucaramanga,BUCARAMANGA,SANTANDER,0,1,False,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7657,2023,4.843900,74.066290,1,3,19,Calle14 13 80,25175,1,5,...,1839,0.932572,Chía,CHÍA,CUNDINAMARCA,0,1,False,True,True
7658,2023,NaN,NaN,1,2,7,Cl 6 a 3 99,25175,1,4,...,1839,0.932572,Chía,CHÍA,CUNDINAMARCA,0,1,False,True,True
7659,2023,4.857498,74.055293,1,7,5,Calle 10 4 48 capincito,25175,1,5,...,1839,0.932572,Chía,CHÍA,CUNDINAMARCA,0,1,False,True,False
7660,2023,4.853827,74.056633,2,6,35,Cea 3 6 20,25175,1,3,...,1839,0.932572,Chía,CHÍA,CUNDINAMARCA,0,1,False,True,True


crear variables dummies para la pregunta 33

In [64]:
def create_dummies_P33 (x):
    if x == np.nan:
        return False
    else:
        return True

df_data_1["frec_uso_si"]=df_data_1["P33"].apply(create_dummies_P33)
df_data_1 = df_data_1.drop(["P33"], axis=1)
df_data_1

,AÑO,GPSLAT,GPSLONG,SECTOR,SECCION,MANZANA,DIRECCION_FILTRO,DANE5,PB1,PERSONAS,...,TASA_INTERNET_ICFES,MUNICIPIO_NOMBRE,Nombre Municipio,Nombre Departamento,Servicios_Telecomunicaciones_No,Servicios_Telecomunicaciones_Si,Dispositivos_hogar_No,conexion_hogar_si,interrupciones_si,frec_uso_si
0,2018,10.952493,-74.777018,2201,8,3,Kr 14 # 21 105,08001,1,1,...,0.698392,Barranquilla,BARRANQUILLA,ATLÁNTICO,1,0,True,False,False,True
1,2018,NaN,NaN,203,1,13,kra 46A # 110_64,05001,1,5,...,0.809559,Medellín,MEDELLÍN,ANTIOQUIA,0,1,True,False,False,True
2,2018,NaN,NaN,203,1,13,Kra 46A # 110_60,05001,1,4,...,0.809559,Medellín,MEDELLÍN,ANTIOQUIA,0,1,True,False,False,True
3,2018,6.299582,-75.550590,203,2,2,kra 109 201 _28,05001,1,2,...,0.809559,Medellín,MEDELLÍN,ANTIOQUIA,0,1,False,True,False,True
4,2018,7.090175,-73.145032,798,2,25,Cll 58 47 aw 8,68001,1,4,...,0.830748,Bucaramanga,BUCARAMANGA,SANTANDER,0,1,False,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7657,2023,4.843900,74.066290,1,3,19,Calle14 13 80,25175,1,5,...,0.932572,Chía,CHÍA,CUNDINAMARCA,0,1,False,True,True,True
7658,2023,NaN,NaN,1,2,7,Cl 6 a 3 99,25175,1,4,...,0.932572,Chía,CHÍA,CUNDINAMARCA,0,1,False,True,True,True
7659,2023,4.857498,74.055293,1,7,5,Calle 10 4 48 capincito,25175,1,5,...,0.932572,Chía,CHÍA,CUNDINAMARCA,0,1,False,True,False,True
7660,2023,4.853827,74.056633,2,6,35,Cea 3 6 20,25175,1,3,...,0.932572,Chía,CHÍA,CUNDINAMARCA,0,1,False,True,True,True


Se calcula una métrica que captura la relación entre los hogares con internet del municipio y la cantidad de hogares del municipio

In [65]:
df_data_1['dens_int'] = 1- (df_data_1['HOGARES_INTERNET']/df_data_1['POBLACIÓN_ICFES'])

# Implementación modelo

In [66]:
# ============================================
# 0) Imports y utilidades
# ============================================
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

def load_data(path="df_data_1"):
    df = df_data_1
    # Asegurar columnas esperadas
    missing = [c for c in FEATURES if c not in df.columns]
    if missing:
        raise ValueError(f"Faltan columnas en data_procesada: {missing}")

    # Coerción a numérico/bool
    for c in FEATURES:
        # Si es binario, mapear True/False/NaN a 1/0
        if c != "dens_int":
            df[c] = df[c].astype("float").astype("Int64")  # por si viene como 0/1/NaN
            df[c] = df[c].fillna(0).astype(int)
        else:
            df[c] = pd.to_numeric(df[c], errors="coerce")

    # Imputación mínima para dens_int
    if df["dens_int"].isna().any():
        df["dens_int"] = df["dens_int"].fillna(df["dens_int"].median())

    return df

In [68]:
# --- 1) Variables para clustering  ---
FEATURES = [
    "Servicios_Telecomunicaciones_No",
    "Servicios_Telecomunicaciones_Si",
    "Dispositivos_hogar_No",
    "conexion_hogar_si",
    "interrupciones_si",
    "frec_uso_si",
    "dens_int",
]

# ============================================
# 1) Cargar datos
# ============================================

df = load_data(df_data_1)

In [69]:
# --- 2) Preparar matriz de entrenamiento ---
X = df[FEATURES].copy()
scaler = StandardScaler()
X[["dens_int"]] = scaler.fit_transform(X[["dens_int"]])

# --- 3) Entrenar K-means con k=12 (configuración seleccionada) ---
kmeans = KMeans(n_clusters=12, n_init=50, random_state=123)
labels = kmeans.fit_predict(X)

# Añadir columna de cluster al dataframe original
df_lab = df.copy()
df_lab["cluster"] = labels

In [70]:
# --- 4) Normalización para empatar con shapefile ---
def norm(s: pd.Series) -> pd.Series:
    return (s.astype(str).str.upper()
             .str.normalize("NFKD").str.encode("ascii","ignore").str.decode("utf-8")
             .str.replace(r"[^A-Z0-9\s]", " ", regex=True)
             .str.replace(r"\s+", " ", regex=True)
             .str.strip())

df_lab["dep_norm"] = norm(df_lab["Nombre Departamento"])

In [71]:
df_lab.to_csv("PRUEBA.csv", index=False, encoding="utf-8")